# Perform regionalization when no parameter set is available

Here we call the Regionalization WPS service to provide estimated streamflow (best estimate and ensemble) at an ungauged site using three pre-calibrated hydrological models and a large hydrometeorological database with catchment attributes (Extended CANOPEX). Multiple regionalization strategies are allowed. 

In [ ]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import json
import os

# Set environment variable RAVEN_WPS_URL to "http://localhost:9099" to run on the default local server
url = os.environ.get("RAVEN_WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")
wps = WPSClient(url)

In [ ]:
# Get the documentation for the method's usage:
help(wps.regionalisation)

In [ ]:
# Forcing files. This file should only contain weather data (tmin, tmax, rain, snow, pet (if desired), etc. 
# No streamflow is required. This is a link to a string, but you can submit a string to your netcdf file directly.
ts = str(TESTDATA['raven-hmets-nc-ts'])

# Model configuration parameters
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    name='Saumon',
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    method='PS', # One of the methods described above
    model_name='HMETS', # One of the three models are allowed: HMETS, GR4JCN and MOHYSE
    min_nse=0.7, # Minimumcalibration NSE required to be considered a donor (for selecting good donor catchments)
    ndonors=5, # Number of donors we want to use. Usually between 4 and 8 is a robust number.
    properties=json.dumps({'latitude':54.4848, 'longitude':-123.3659, 'forest':0.4}),
    )

# Let's call the model with the timeseries, model parameters and other configuration parameters
resp = wps.regionalisation(ts=ts, **config)

In [ ]:
# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally. 
[hydrograph, ensemble] = resp.get(asobj=True)

The `hydrograph` and `ensemble` outputs are netCDF files storing the time series. These files are opened by default using `xarray`, which provides convenient and powerful time series analysis and plotting tools. 

In [ ]:
hydrograph.q_sim

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

hydrograph.q_sim.plot()

In [ ]:
print("Max: ", hydrograph.q_sim.max())
print("Mean: ", hydrograph.q_sim.mean())
print("Monthly means: ", hydrograph.q_sim.groupby('time.month').mean(dim='time'))

Now we can also see the results coming from the 5 donors using the 'ensemble' variable

In [ ]:
# Plot the simulations from the 5 donor parameter sets
ensemble.q_sim.isel(nbasins=0).plot.line(hue='realization')

In [ ]:
# You can also obtain the data in netcdf format directly by changing asobj to False:
[hydrograph, ensemble] = resp.get(asobj=False)
print(hydrograph)
print(ensemble)